# Introduction aux décorateurs en Python


## Qu'est-ce qu'un décorateur?

Les décorateurs en Python sont une fonctionnalité puissante qui permet de modifier ou d'étendre le comportement d'autres fonctions ou méthodes.


## À quoi servent les décorateurs?

1. **Réutilisation de code**: Les décorateurs sont souvent utilisés pour extraire du code commun qui peut être réutilisé, afin de ne pas répéter le même code dans plusieurs endroits.
2. **Séparation des préoccupations**: Ils vous permettent de séparer les préoccupations et d'ajouter des fonctionnalités, comme le journalisation (logging) ou la mise en cache, sans modifier le code de la fonction elle-même.
3. **Paramétrage des fonctions**: Les décorateurs peuvent également être utilisés pour modifier les arguments passés à une fonction, ou pour modifier sa valeur de retour (peu recommandé pour des questions de maintenabilité).

## Deux types de décorateurs

### Décorateur simple (sans argument)

In [1]:
import time

def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        print(f"Temps d'exécution de {func.__name__}: {end_time - start_time} secondes")
        return result
    return wrapper

@timing_decorator 
def slow_function():
    time.sleep(2)

slow_function()

Temps d'exécution de slow_function: 2.0004734992980957 secondes


### Décorateur avec arguments

In [3]:
def logging_decorator(prefix):
    def decorator(func):
        def wrapper(*args, **kwargs):
            print(f"{prefix} Avant l'exécution de {func.__name__}")
            result = func(*args, **kwargs)
            print(f"{prefix} Après l'exécution de {func.__name__}")
            return result
        return wrapper
    return decorator

@logging_decorator("TOTO")
def my_function():
    print("Exécution de la fonction")

my_function()

TOTO Avant l'exécution de my_function
Exécution de la fonction
TOTO Après l'exécution de my_function


## Utilisation des décorateurs pour la gestion d'erreur


In [7]:
import time
import random

def retry_decorator(max_retries=3, delay=1):
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(max_retries):
                try:
                    return func(*args, **kwargs)
                except ValueError as e:
                    print(f"Erreur: {e}. Réessayer après {delay} secondes...")
                    time.sleep(delay)
        return wrapper
    return decorator

@retry_decorator(max_retries=10, delay=2)
def unreliable_function():
    if random.random() < 0.7:
        raise ValueError("Une erreur s'est produite")
    return "Succès!"

print(unreliable_function())

Erreur: Une erreur s'est produite. Réessayer après 2 secondes...
Erreur: Une erreur s'est produite. Réessayer après 2 secondes...
Succès!


Il existe un package python [retry](https://github.com/invl/retry) qui implemente ce genre de fonctionnalité de manière plus générique, mais si vous voulez gérer plus finement vos erreurs les décorateurs sont plus efficaces !

# Exemples de décorateurs existants

## Utilisation des décorateurs dans la bibliothèque standard de Python

### functools.lru_cache

Le décorateur `lru_cache` de la bibliothèque standard `functools` est utilisé pour mémoiser les appels de fonction. Cela signifie qu'il stocke les résultats des appels de fonction dans un cache, de sorte que lors d'appels ultérieurs avec les mêmes arguments, les résultats sont renvoyés à partir du cache plutôt que d'exécuter à nouveau la fonction.

In [8]:
def naive_fibo(n):
    if n < 2:
        return n
    return naive_fibo(n-1) + naive_fibo(n-2)

%timeit naive_fibo(35)

1.3 s ± 8.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
from functools import lru_cache


@lru_cache(maxsize=None)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

%timeit fibonacci(35)
%timeit fibonacci(100)

30.2 ns ± 0.296 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
34.6 ns ± 0.128 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### functools.wraps

Le décorateur `wraps` dans le module `functools` est utilisé dans la définition de décorateurs personnalisés pour s'assurer que les métadonnées de la fonction d'origine sont conservées après avoir été décorées.

In [13]:
from functools import wraps

def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        print("Avant l'appel de la fonction")
        result = f(*args, **kwargs)
        print("Après l'appel de la fonction")
        return result
    return wrapper

@my_decorator
def example():
    """Ceci est une fonction d'exemple"""
    print("Exemple de fonction")

example.__doc__


"Ceci est une fonction d'exemple"

## Utilisation dans des bibliothèques tierces

### Numpy

Numpy a un décorateur `vectorize`qui convertit une fonction Python ordinaire acceptant des scalaires en une fonction "vectorisée" qui peut agir sur des tableaux entiers.

In [15]:
import numpy as np

@np.vectorize
def add(x, y):
    return x + y

result = add(np.array([1, 2, 3]), np.array([4, 5, 6]))
repr(result)


'array([5, 7, 9])'

### pytest

La bibliothèque de tests pytest utilise des décorateurs pour marquer les tests à sauter ou pour lesquels une erreur est attendue :

In [ ]:
import pytest

# sauter le test
@pytest.mark.skip(reason="Il n'est pas possible de tester cette fonctionnalité pour l'instant")
def test_example():
    assert False

# il est attendu que le test soit invalide
@pytest.xfail(reason="Cette fonctionnalité n'est pas encore implémentée")
def test_feature_x():
    assert False

Le décorateur `@pytest.fixture` est utilisé pour créer des fixtures. Les fixtures sont utilisées pour fournir un état fixe et fiable à l'exécution des tests. Cela peut être pour configurer un environnement de test, créer une ressource, une connexion à une base de données, etc.

In [ ]:
import pytest

@pytest.fixture
def sample_data():
    return {"key1": "value1", "key2": "value2"}

def test_example_1(sample_data):
    assert "key1" in sample_data

def test_example_2(sample_data):
    assert sample_data["key2"] == "value2"

# FIN

Si vous avez des questions ou si vous souhaitez partager d'autres exemples intéressants d'utilisation des décorateurs, mettez